In [4]:
import os.path
import re

from common.consts import PATTERN_FILE_FORMAT__MSI, PATTERN_TYPE__BROADCAST, PATTERN_TYPE__BEAMFORMING_ELEMENT, PATTERN_TYPE__BEAMSWITCHING_SERVICE
from common.beamforming_antenna_generator import BeamformingAntennaGenerator
from common.re_filter import ReFilter
from common.pattern_name_param_extractor import PatternNameParamExtractor
from common.pattern_payload_param_extractor import PatternPayloadParamExtractor
from common.pattern_name_param_selector import PatternNameParamSelector

In [5]:
generator = BeamformingAntennaGenerator({
    # General parameters
    'src_folder': r'D:\Cell Planning\Antenna model processing\Library\mMIMO\mMIMO Antennas\Nokia\AQQA Full_eTilt_Offset',
    'pattern_file_format': PATTERN_FILE_FORMAT__MSI,
    'version': '7.4',
    'filename': 'AQQA_32T32R.pafx',
    'name': 'AQQA MIMO 32T32R 96 ELEM',
    'type': 'Cellular',
    'comment': 'AQQA MIMO 32T32R 96 ELEM',
    'manufacturer': 'Nokia',
    'cost': 0,
    'cost_unit': 'USD',
    'length_cm': 61,
    'width_cm': 38,
    'depth_cm': 10,
    'weight_kg': 27,
    'wind_load_factor': 587,
    'supp_elec_tilt': True,
    'supp_elec_azimuth': False,
    'supp_elec_beamwidth': False,
    'cont_adj_elec_tilt': False,
    
    # Source file filter
    'src_file_re_filter': ReFilter(
        allow=[
            '.*Optimized.*(SSB|RefBeam|SsbBeam).*\.msi$',
        ],
        deny=[
            '.*TypeApproval.*',
            '.*3GPP.*',
            '.*PatternEnvelope.*',
        ],
    ),
    
    # Parameter extractors
    'pattern_name_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='(?P<cg>.+)\..{3}$',
    ),
    'scenario_extractor': PatternNameParamExtractor(
        extract_re='.*-(?P<cg>\d+deg.+)-(Envelope|RefBeam|SsbBeam).*',
        post_capture_proc=lambda r: re.sub('-(p|n)\d+-a(p|n)\d+-', '-', r),
    ),
    'v_port_name_extractor': PatternNameParamExtractor(
        extract_re='.*-(?P<cg>\d+deg.+)-(Envelope|RefBeam|SsbBeam).*',
        post_capture_proc=lambda r: re.sub('-(p|n)\d+-a(p|n)\d+-', '-', r),
    ),
    'pattern_type_extractor': PatternNameParamExtractor(
        path_part='basename',
        pre_capture_proc=lambda r: r.lower(),
        post_capture_proc=lambda r: PATTERN_TYPE__BROADCAST if 'envelope' in r else PATTERN_TYPE__BEAMSWITCHING_SERVICE,
    ),
    'center_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(float(payload.header['FREQUENCY']))
    ),
    'min_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(float(payload.header['FREQUENCY']) - 100)
    ),
    'max_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(float(payload.header['FREQUENCY']) + 100)
    ),
    'electrical_tilt_extractor': PatternNameParamExtractor(
        path_part='basename',
        pre_capture_proc=lambda r: r.lower(),
        extract_re='.*-(?P<cg>(p|n)\d+)-.*',
        post_capture_proc=lambda r: int(r.replace('p', '').replace('n', '-')),
    ),
    'polarization_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 'Vertical',
    ),
    'polarization_type_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: None,
    ),
    'v_port_number_of_ports_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 1,
    ),
    'horiz_number_of_elements_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4,
    ),
    'horiz_sep_dist_cm_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4.3,
    ),
    'vert_number_of_elements_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4,
    ),
    'vert_sep_dist_cm_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 17.7,
    ),
    'beamswitching_service_name_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 'PDSCH' if 'RefBeam' in r else 'SSB' if 'SsbBeam' in r else 'None',
    ),
    'beamswitching_horiz_angle_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: payload.horiz_pap_pattern.get_boresight_deg(),
    ),
    'beamswitching_vert_angle_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: payload.vert_pap_pattern.get_boresight_deg(),
    ),
    
    # Parameter selectors
    'scenario_selector': None,
    'v_port_name_selector': None,
})

In [6]:
extracted_tags = generator.list_extracted_tags(detailed=False)  


Extracted tags list

[scenario]:
   1.  120degAzOp-#1...........18 items
   2.  120degAzOp-#2...........33 items
   3.  120degAzOp-#2#2.........21 items
   4.  120degAzOp-#3#1.........21 items
   5.  120degAzOp-#3#3.........31 items
   6.  120degAzOp-#3#3#2.......41 items
   7.  120degAzOp-#3#3#2b......41 items
   8.  120degAzOp-#4...........63 items
   9.  120degAzOp-#4#2.........31 items
  10.  120degAzOp-#4#4.........41 items
  11.  120degAzOp-#5#1.........31 items
  12.  120degAzOp-#5#3.........41 items
  13.  120degAzOp-#6...........93 items
  14.  120degAzOp-#6#2.........41 items
  15.  120degAzOp-#8...........123 items
  16.  40degAzOp-#2#2#2#2......41 items
  17.  90degAzOp-#1............18 items
  18.  90degAzOp-#2............33 items
  19.  90degAzOp-#2#2..........21 items
  20.  90degAzOp-#2#2#2#2......41 items
  21.  90degAzOp-#3#1..........21 items
  22.  90degAzOp-#3#3..........31 items
  23.  90degAzOp-#3#3#2........41 items
  24.  90degAzOp-#3#3#2b.......41 items
  25.

In [7]:
generator.generate(os.path.abspath('.\output'))


Scenario > Virutal port > Virutal band > Pattern assignments
{
  "40degAzOp-#2#2#2#2": {
    "uid": "2",
    "name": "40degAzOp-#2#2#2#2",
    "horiz_number_of_elements": 4,
    "horiz_sep_dist_cm": 4.3,
    "vert_number_of_elements": 4,
    "vert_sep_dist_cm": 17.7,
    "v_ports": {
      "40degAzOp-#2#2#2#2": {
        "uid": "3",
        "name": "40degAzOp-#2#2#2#2",
        "number_of_ports": 1,
        "polarization": "Vertical",
        "polarization_type": null,
        "v_bands": {
          "3400-3600": {
            "min_freq": 3400,
            "max_freq": 3600,
            "supp_elec_tilt": true,
            "supp_elec_azimuth": false,
            "supp_elec_beamwidth": false,
            "cont_adj_elec_tilt": false,
            "broadcast_patterns": [
              "AQQA-Optimized-40degAzOp-p00-ap07-#2#2#2#2-Envelope_L1-SSB"
            ],
            "electrical_controller_name": "Controller 1",
            "use_elec_params_for_bs_service_patterns": true,
            "be